In [ ]:
#download the data and save it to poem.txt
!wget https://www.gutenberg.org/files/1661/1661-0.txt -O book.txt

--2025-03-07 00:48:00--  https://www.gutenberg.org/files/1661/1661-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607504 (593K) [text/plain]
Saving to: ‘book.txt’

book.txt            100%[===================>] 593.27K   511KB/s    in 1.2s    

2025-03-07 00:48:02 (511 KB/s) - ‘book.txt’ saved [607504/607504]



In [ ]:
#read the file in text string
text = open('book.txt', 'r', encoding='utf-8').read()
text = text.lower()

In [ ]:
#create list of sentences
sentences = text.split('\n')

In [ ]:
#import dependencies to preprocess the text data and making sequences
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
#initialize the tokenizer, which can work char by char
tokenizer = Tokenizer(oov_token='<UNK>')

In [ ]:
tokenizer.fit_on_texts(sentences)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

8923

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)

In [ ]:
input_sequences = []
for sequence in sequences:
  for i in range(1, len(sequence)):
    n_gram_sequence = sequence[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
print(input_sequences[0], input_sequences[1], input_sequences[2], input_sequences[3])

[4776, 158] [4776, 158, 331] [4776, 158, 331, 886] [4776, 158, 331, 886, 5]


In [ ]:
#find the maximum length among sequences
max_seq_len = max([len(seq) for seq in input_sequences])
max_seq_len

20

In [ ]:
#we will keep the last value of the sequence as our target label and all values before that as input to sequence model

In [ ]:
#pad the sequences to ensure that they are all of same length
padded_sequences = pad_sequences(input_sequences, maxlen = max_seq_len)

In [ ]:
print(padded_sequences[0], padded_sequences[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0 4776  158] [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 4776  158  331]


In [ ]:
import numpy as np
padded_sequences = np.array(padded_sequences)

In [ ]:
print(len(padded_sequences[0]))
print(len(padded_sequences[1]))

20
20


In [ ]:
#prepare training sequences and labels
x = padded_sequences[:, : -1]
labels = padded_sequences[:, -1]

In [ ]:
labels.shape

(101523,)

In [ ]:
#to one hot encode the labels
y = tf.keras.utils.to_categorical(labels, num_classes=vocab_size)

In [ ]:
x.shape

(101523, 19)

In [ ]:
#x = x.reshape(x.shape[0], x.shape[1], 1)

In [ ]:
y.shape

(101523, 8923)

In [ ]:
#import dependencies for defining the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
#define and compile the model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_seq_len-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Again try

# Define and compile the model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_seq_len-1))  # Input length specified
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size, activation='softmax'))

# Define optimizer
adam = Adam(learning_rate=0.01)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])

# Build model explicitly to ensure parameters are counted
model.build(input_shape=(None, max_seq_len-1))

# Print model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 19, 100)             │         892,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 512)                 │         731,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8923)                │       4,577,499 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,200,935 (23.65 MB)

 Trainable params: 6,200,935 (23.65 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#initialize the callback for early stopping the training if there is not at least 1% improvement in the accuracy
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'acc', min_delta=0.01)

In [ ]:
model.fit(x, y, epochs=50, verbose=1, batch_size=512, callbacks=[es])

Epoch 1/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - acc: 0.0631 - loss: 6.7026
Epoch 2/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - acc: 0.1395 - loss: 5.3380
Epoch 3/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - acc: 0.1775 - loss: 4.7172
Epoch 4/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - acc: 0.2092 - loss: 4.1696
Epoch 5/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - acc: 0.2595 - loss: 3.6624
Epoch 6/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - acc: 0.3146 - loss: 3.2310
Epoch 7/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - acc: 0.3678 - loss: 2.8568
Epoch 8/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - acc: 0.4187 - loss: 2.5511
Epoch 9/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - acc: 0.4643 - loss: 2.3115
Epoch 10/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - acc: 0.5045 - loss: 2.0929
Epoch 11/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - acc: 0.5362 - loss: 1.9277
Epoch 12/50
199/199 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - acc: 0.5613 - loss: 1.7953
Epoch 

In [ ]:
#Time to become storyteller!

seed_text = "I could not help laughing at the ease with which he explained his process of deduction"
# seed_text = "He is very good at explaining things"

next_words = 100

for _ in range(next_words):
    sequence = tokenizer.texts_to_sequences([seed_text])
    padded = pad_sequences(sequence, maxlen=max_seq_len-1)

    # Use model.predict() and get the class with argmax
    predicted = np.argmax(model.predict(padded, verbose=0))

    output_word = ''
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

    seed_text += ' ' + output_word

print(seed_text)


I could not help laughing at the ease with which he explained his process of deduction “when i thought that i know i was at my wit’s end ” he “g” and “g” and “g” as i “g” it well i am myself one of the “g” of the “g” ” i remarked “for it then it seemed to me that the facts should i have a man “g” whom i have already in the second “g” ” i asked ‘the “g” and i understand that you have seen an opinion upon the second floor “g” ” he “g” “the “g” is a “g” book and a “g” heavy stone had been behind it and his


In [ ]:
#let's look at how loss and accuracy changed while training
import matplotlib.pyplot as plt
history = model.history
acc = history.history['acc']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

KeyError: 'acc'